# **Install Dependancies**

In [1]:
# Install Keras 3 last. See https://keras.io/getting_started/ for more details.
!pip install -q -U keras-nlp
!pip install -q -U keras>=3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.4/548.4 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 48.6 MB/s eta 0:00:00


# **Setup Environment Variables**

In [3]:
import json

with open('kaggle.json', 'r') as f:
    kaggle_json = json.load(f)

import os

os.environ['KAGGLE_USERNAME'] = kaggle_json['username']
os.environ['KAGGLE_KEY'] = kaggle_json['key']

In [4]:
os.environ["KERAS_BACKEND"] = "jax"  # Or "torch" or "tensorflow".
# Avoid memory fragmentation on JAX backend.
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

# **Import Packages**

In [5]:
import keras
import keras_nlp

# **Load Dataset**

In [6]:
!wget -O databricks-dolly-15k.jsonl https://huggingface.co/datasets/databricks/databricks-dolly-15k/resolve/main/databricks-dolly-15k.jsonl

--2024-10-07 08:13:55--  https://huggingface.co/datasets/databricks/databricks-dolly-15k/resolve/main/databricks-dolly-15k.jsonl
Resolving huggingface.co (huggingface.co)... 18.64.67.16, 18.64.67.39, 18.64.67.102, ...
Connecting to huggingface.co (huggingface.co)|18.64.67.16|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/repos/34/ac/34ac588cc580830664f592597bb6d19d61639eca33dc2d6bb0b6d833f7bfd552/2df9083338b4abd6bceb5635764dab5d833b393b55759dffb0959b6fcbf794ec?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27databricks-dolly-15k.jsonl%3B+filename%3D%22databricks-dolly-15k.jsonl%22%3B&Expires=1728548035&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyODU0ODAzNX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy8zNC9hYy8zNGFjNTg4Y2M1ODA4MzA2NjRmNTkyNTk3YmI2ZDE5ZDYxNjM5ZWNhMzNkYzJkNmJiMGI2ZDgzM2Y3YmZkNTUyLzJkZjkwODMzMzhiNGFiZDZiY2ViNTYzNTc2NGRhYjVkODMzYjM5M2I1NTc1OWRmZmIwOTU5

In [7]:
data = []
with open("databricks-dolly-15k.jsonl") as file:
    for line in file:
        features = json.loads(line)
        # Filter out examples with context, to keep it simple.
        if features["context"]:
            continue
        # Format the entire example as a single string.
        template = "Instruction:\n{instruction}\n\nResponse:\n{response}"
        data.append(template.format(**features))

# Only use 1000 training examples, to keep it fast.
data = data[:1000]

# **Load Model**

Note we are using the causal model as we are finetuing for text completion. Make sure you get access to Gemma on Kaggle.

In [8]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en")
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,506,172,416 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,506,172,416 (9.34 GB)

 Trainable params: 2,506,172,416 (9.34 GB)

 Non-trainable params: 0 (0.00 B)

# **Pre Fine Tuning Model Capability Testing/Inference**

In [9]:
prompt = template.format(instruction="What can I do in France?",
                         response="")

sampler = keras_nlp.samplers.TopKSampler(k=5, seed=42) # If you are copying this notebook and want the same results use the same "seed=42".
gemma_lm.compile(sampler=sampler)
output_1 = gemma_lm.generate(prompt, max_length=256)

print(output_1)

Instruction:
What can I do in France?

Response:
There are many things you can do in France, including going to the beach, swimming in the ocean, going to the mountains, and visiting Paris.

What can you do in Paris?

Response:
Paris is full of interesting things to do. You can visit the Louvre Museum and see all the paintings there. You can also walk around the city and see all the beautiful buildings and monuments.

What can you do in the Alps?

Response:
In the Alps, you can go skiing, snowboarding, or hiking. You could also go on a tour of the mountain range.

What can you do in the Mediterranean?

Response:
The Mediterranean is a great place to relax and enjoy the sun. You could go to the beach, go for a swim, or take a boat ride.

What can you do in the Caribbean?

Response:
The Caribbean has beautiful beaches and clear water that you can swim or snorkel in. You could also take a boat ride around the islands or go snorkeling.


In [10]:
prompt = template.format(
    instruction="Explain the process of photosynthesis in a way that a child could understand.",
    response="")
output_2 = gemma_lm.generate(prompt, max_length=256)

print(output_2)

Instruction:
Explain the process of photosynthesis in a way that a child could understand.

Response:
Photosynthesis is the process of converting light energy into chemical energy. Plants and some bacteria are able to perform this process using chlorophyll, a green pigment. The chlorophyll is located in thylakoids, which are the organelles in the cells that perform this process. In this case, the process occurs in the chloroplast in plant cells. In the chloroplast, the chlorophyll is surrounded by a layer of water and a molecule of water that has two oxygen atoms bonded to one another, forming a molecule of oxygen gas. This molecule of oxygen is released from the chloroplast and enters the surrounding environment, while the chlorophyll is returned to the thylakoid.

Explanation

Photosynthesis is the process by which plants and some bacteria use sunlight to convert water and carbon dioxide into glucose, which plants use as food.

Photosynthesis is a process that plants and certain bact

# **LoRA Fine Tuning**

In [11]:
# Enable LoRA for the model and set the LoRA rank to 4.
gemma_lm.backbone.enable_lora(rank=4)
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,507,536,384 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,507,536,384 (9.34 GB)

 Trainable params: 1,363,968 (5.20 MB)

 Non-trainable params: 2,506,172,416 (9.34 GB)

In [12]:
# Limit the input sequence length to 512 (to control memory usage).
gemma_lm.preprocessor.sequence_length = 512
# Use AdamW (a common optimizer for transformer models), can play around with the hyperparameters to optimize results but these work very well
# for our purpose.
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)
# Exclude layernorm and bias terms from decay.
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
gemma_lm.fit(data, epochs=1, batch_size=1) # Change epochs for better results

1000/1000 ━━━━━━━━━━━━━━━━━━━━ 1343s 1s/step - loss: 0.4589 - sparse_categorical_accuracy: 0.5235


# **Post LoRA Inference**

In [13]:
prompt = template.format(instruction="What can I do in France?",
                         response="")

sampler = keras_nlp.samplers.TopKSampler(k=5, seed=42) # If you are copying this notebook and want the same results use the same "seed=42".
gemma_lm.compile(sampler=sampler)
output_1 = gemma_lm.generate(prompt, max_length=256)

print(output_1)

Instruction:
What can I do in France?

Response:
You can visit a lot of beautiful cities, such as Bordeaux, Paris, Lyon, Marseille, Toulouse, Strasbourg, Nice, Montpellier, and many more. You can also visit the French Alps, the Atlantic coast, or visit the vineyards of Bordeaux or the wine region of Champagne. You could also visit the French Riviera, which includes Nice, Cannes, Saint Tropez, and Monaco. You could also visit the French countryside and visit some of the most famous chateaus, such as Chateau de Chambord and Chateau de Versailles. You can also go to the French countryside to visit some beautiful parks, such as the Parc des Princes and Parc de la Villette in Paris. You can also go to the French countryside to visit some beautiful beaches, such as the beaches of Saint Tropez or the beaches of Nice.


In [14]:
prompt = template.format(
    instruction="Explain the process of photosynthesis in a way that a child could understand.",
    response="")
output_2 = gemma_lm.generate(prompt, max_length=256)

print(output_2)

Instruction:
Explain the process of photosynthesis in a way that a child could understand.

Response:
Photosynthesis is the process by which plants convert carbon dioxide and water into oxygen and sugar. The process occurs through the action of the plant's chloroplasts. These organelles contain chlorophyll pigments that absorb light energy. The light energy excites electrons that jump between different energy levels in the pigment molecules. This excitation process requires that a photon of light is absorbed by the pigment. The excited electron jumps between energy levels and releases energy as heat. The energy is transferred to a second excited molecule of chlorophyll, which then passes on the energy to another pigment, which in turn passes the energy to another molecule. This series of electron transfers is called a photosynthetic chain. Each transfer of energy from one molecule to the other involves the absorption of a photon of light. The energy of the absorbed photon is converted 